In [10]:
# !pip install selenium

In [11]:
# !pip install webdriver-manager

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time

# -----
game_result = []
# -----

In [15]:
# Chrome Browser와 Chrome Driver Version 확인
chrome_options = webdriver.ChromeOptions()

# 요청
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
url = 'https://www.koreabaseball.com/Schedule/Schedule.aspx'
driver.get(url)

time.sleep(2)

WebDriverException: Message: Can not connect to the Service /Users/sori/.wdm/drivers/chromedriver/mac64/125.0.6422.141/chromedriver-mac-arm64/chromedriver


In [14]:
# 키움 탭 선택
driver.find_element(By.XPATH, '//*[@id="boxList"]/ul/li[8]/a').click()

# 연도 선택 박스
year_box = driver.find_element(By.CLASS_NAME, 'date').find_element(By.ID, 'ddlYear')
year_lists = year_box.text.split('\n')[:3]
year_lists.reverse()

# 월별 선택 박스
month_box = driver.find_element(By.CLASS_NAME, 'date').find_element(By.ID, 'ddlMonth')
month_lists = month_box.text.split('\n')

# year_lists = ['2022','2023']
# month_lists = ['10','11','12']
# year_lists = ['2022','2023']
# month_lists = ['9','10']

# 날짜입력 (년/월)
for year in year_lists :
    year_box.send_keys(year)
    time.sleep(1)

    for month in month_lists :
        month_box.send_keys(month)
        time.sleep(1)

        try :
            # 데이터 한줄, tbody에서 tr 가져오기
            tr_lists = driver.find_element(By.XPATH, '//*[@id="tblScheduleList"]/tbody').find_elements(By.TAG_NAME, 'tr')
            
            print(len(tr_lists))

            # if tr_lists[0].text == '데이터가 없습니다.' :
            #     print('-- 데이터가 없습니다 --')
            #     continue

            game_dict = {}

            for tr in tr_lists :
                print(year, month)

                print('tr >>>>', tr.text)

                # 날짜에 해당하는 경기의 url
                try :
                    review_url = ''
                    review_url = tr.find_element(By.TAG_NAME, 'a').get_attribute('href')
                except :
                    print('url error, 다음 경기 -----')
                    continue

                game_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
                game_driver.get(review_url)

                time.sleep(1.5)

                # 구장 위치
                stadium = game_driver.find_element(By.ID, 'txtStadium').text

                # 경기 날짜
                date = game_driver.find_element(By.ID, 'lblGameDate').text

                # 시작 시간
                start_time = game_driver.find_element(By.ID, 'txtStartTime').text[5:]

                # 종료 시간
                end_time = game_driver.find_element(By.ID, 'txtEndTime').text[5:]

                # 진행한 시간
                run_time = game_driver.find_element(By.ID, 'txtRunTime').text[7:]

                # away팀 투수 테이블
                away_pitcher = game_driver.find_element(By.ID, 'tblAwayPitcher').find_element(By.TAG_NAME, 'tbody').find_element(By.TAG_NAME, 'tr')
                away_pitcher_name = away_pitcher.find_elements(By.TAG_NAME, 'td')[0].text

                # home팀 투수 테이블
                home_pitcher = game_driver.find_element(By.ID, 'tblHomePitcher').find_element(By.TAG_NAME, 'tbody').find_element(By.TAG_NAME, 'tr')
                home_pitcher_name = home_pitcher.find_elements(By.TAG_NAME, 'td')[0].text

                # time.sleep(1) 

                # dict 구성
                game_dict = {
                    '날짜' : date,
                    '시작시간' : start_time,
                    '종료시간' : end_time,
                    '경기시간' : run_time,
                    '홈팀 선발' : home_pitcher_name,
                    '어웨이팀 선발' : away_pitcher_name,
                    '구장' : stadium
                }

                print(date,stadium)

                # array append
                game_result.append(game_dict)

                time.sleep(0.5)
                game_driver.quit()

        except :
            print('데이터 없음, 다음월 -----')
            continue

KeyboardInterrupt: 